In [10]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import datetime as dt
import math as mt
import IPython
import IPython.display
from sklearn.decomposition import TruncatedSVD 
from scipy.spatial import distance
import scipy.stats as stats
from tqdm import tqdm
from numba import jit, cuda

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [11]:
# 1~6, 9~12
low_pressure_fee = [
    [0,200,910,88.3],
    [201,400,1600, 182.9],
    [401,float('inf'),7300, 275.6]
]
# 7~8
low_pressure_fee_summer = [
    [0,300,910,88.3],
    [301,450,1600,182.9],
    [451,float('inf'),7300,275.6]
]
# 1~6, 9~12
high_pressure_fee = [
    [0,200,730,73.3],
    [201,400,1260,142.3],
    [400,float('inf'),6060,210.6]
]
# 7~8
high_pressure_fee_summer = [
    [0,300,730,73.3],
    [301,450,1260,142.3],
    [451,float('inf'),6060,210.6]
]

general_fee = {
    "저압": [6160, 100.7, 60.2, 87.3],
    "고압 A": [
        [7170, 110.9, 66.9, 98.6],
        [8230, 106.9, 62.6, 93.3]
    ],
    "고압 B": [
        [7170, 108.8, 65.8, 95.6],
        [8230, 103.5, 60.5, 90.3]
    ]
}

env_fee = 5.3
fuel_fee = 3
VAT = 0.1
FUND = 0.037

class Household:
    def __init__(self,
                name, kwh, contract, contract_name):
        self.name = name
        self.kwh = kwh
        self.contract = contract
        self.contract_name = contract_name
        
    def set_bill(self,
                public_fee):
        self.self_fee = self.elec_bill_vat_fund
        self.public_fee = public_fee
        self.bill = self.self_fee + self.public_fee
        
    @property
    def basic(self):
        fee = None
        for _ in self.contract:
            if _[0] <= self.kwh and _[1] >= self.kwh:
                fee = _[2]
                break
        return fee
    
    @property
    def elec_rate(self):
        kwh = self.kwh
        fee = 0
        for _ in self.contract:
            if kwh <= _[1]:
                fee += (kwh * _[3])
                break
            else:
                kwh -= _[1]
                fee += (_[1] * _[3])
        
        return mt.floor(fee)
        
    @property
    def guarantee(self):
        if self.kwh <= 200:
            if self.contract_name == "종합계약":
                return 4000
            elif self.contract_name == "단일계약":
                return 2500
        else:
            return 0
        
    @property
    def elec_bill(self):
        bill = self.basic + self.elec_rate\
            - self.guarantee + self.env - self.fuel
        
        if bill < 1000:
            return 1000
        else:
            return bill
    
class Public:
    def __init__(self,
                kwh,
                charge_applied,
                contract):
        # 공용설비사용량
        self.kwh = kwh
        # 요금적용전력
        self.charge_applied = charge_applied
        self.contract = contract
        
    @property
    def basic(self):
        return mt.floor(self.charge_applied * self.contract[0])
    
    @property
    def elec_rate(self):
        return mt.floor(self.kwh * self.contract[1])
    
    @property
    def elec_bill(self):
        bill = self.basic + self.elec_rate + self.env - self.fuel
        
        if bill < 1000:
            return 1000
        else:
            return bill
    
@property
def env(self):
    return mt.floor(self.kwh * env_fee)
@property
def fuel(self):
    return mt.floor(self.kwh * fuel_fee)
@property
def vat(self):
    return round(self.elec_bill * 0.1)
@property
def fund(self):
    return mt.floor(self.elec_bill * 0.037 * 0.1) * 10
@property
def elec_bill_vat_fund(self):
    return mt.floor((self.elec_bill + self.vat + self.fund) * 0.1) * 10

Household.env = env
Household.fuel = fuel
Household.vat = vat
Household.fund = fund
Household.elec_bill_vat_fund = elec_bill_vat_fund

Public.env = env
Public.fuel = fuel
Public.vat = vat
Public.fund = fund
Public.elec_bill_vat_fund = elec_bill_vat_fund
    
class ManagementOffice:
    def __init__(self, 
                 month, peaks, households, APT, # datas
                 contract,
                 general_fee_info=None):
        print("[관리사무소] Init.")
        
        self.peaks = peaks
        self.APT = APT
        
        self.contract = contract
        self.select_fee(month, contract, general_fee_info)
        
        if contract == "종합계약":
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee[0],
                        contract_name=contract
                    )
                )

            # 공용설비사용요금
            print("[관리사무소] 공용설비사용요금 계산")
            households_kwh = sum([_.kwh for _ in self.households])
            public_kwh = APT - households_kwh
            max_peak = peak_df['peak (kW)'].max()
            charge_applied = max_peak * (public_kwh / APT)
            
            self.public = Public(
                kwh=public_kwh,
                charge_applied=charge_applied,
                contract=self.fee[1]
            )

            # 가구별 청구서 셋팅
            public_fee = round(self.public.elec_bill_vat_fund / len(self.households) / 10) * 10
            
            print("public fee", public_fee)
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

            # 아파트 청구서 셋팅
            bill = 0
            for household in self.households:
                bill += household.bill
            self.bill = bill
        elif contract == "단일계약":
            # 전체를 하나의 가구 본 상태에서 계약대로 계산식 진행
            # 후에 * len(household) 를 통해 아파트 전체 요금 통지서 확보
            num_household = len(households)
            mean_kwh = APT / num_household

            apart = Household(
                name="아파트",
                kwh=mean_kwh,
                contract=self.fee,
                contract_name=contract
            )
            basic = apart.basic * num_household
            elec_rate = apart.elec_rate * num_household
            env = apart.env * num_household
            fuel = apart.fuel * num_household
            
            elec_bill = basic + elec_rate + env - fuel
            self.bill = mt.floor((elec_bill \
                + round(elec_bill * 0.1) \
                + mt.floor(elec_bill * 0.037 * 0.1) * 10) * 0.1) * 10
            
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee,
                        contract_name=contract
                    )
                )
                
            # 공공설비사용요금
            households_bill = sum(
                [_.elec_bill_vat_fund for _ in self.households]
            )
            print("가구 청구 비용 : {}".format(format(households_bill)))
            self.public_bill = self.bill - households_bill
            
            
            # 가구별 청구서 셋팅
            public_fee = round(self.public_bill / len(self.households))
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

    # 요금제 셋팅 메서드
    def select_fee(self, month, contract, general_fee_info):
            print("[관리사무소] 계약 정보 셋팅")

            if contract == "종합계약":
                household_fee = None
                public_fee = None
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    household_fee = low_pressure_fee
                else:
                    household_fee = low_pressure_fee_summer

                if general_fee_info == None:
                    raise Exception("종합계약은 일반용 전력 정보를 포함해야 합니다.\n")

                tmp_general_fee = None
                if general_fee_info == "저압":
                    tmp_general_fee = general_fee[general_fee_info]
                elif len(general_fee_info) == 2:
                    tmp_general_fee = general_fee[general_fee_info[0]]
                    tmp_general_fee = tmp_general_fee[general_fee_info[1]]
                else: 
                    raise Exception("일반용 전력 설정이 올바르지 않습니다\n"\
                                + "저압, [고압 A, 0 or 1], [고압 B, 0 or 1]")

                if month >= 6 and month <= 8:
                    public_fee = [tmp_general_fee[0], tmp_general_fee[1]]
                elif (month >= 3 and month <= 5) or\
                      (month >= 9 and month <= 10):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[2]]
                elif (month >= 1 and month <= 2) or\
                      (month >= 11 and month <= 12):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[3]]
                self.fee = [household_fee, public_fee]
                
            elif contract == "단일계약":
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    self.fee = high_pressure_fee
                else:
                    self.fee = high_pressure_fee_summer

            print("[관리사무소] 계약 정보 셋팅 완료\n")
            print(self.fee)        
            print("")

In [22]:
def load_excel():
    xlsx = pd.read_excel("datas/datas.xlsx", header=None, skiprows=2, engine="openpyxl")
    
    return xlsx

In [23]:
xlsx = load_excel()

In [24]:
date_df = xlsx[3:][xlsx.columns[1:6]].copy()
household_df = xlsx[xlsx.columns[7:]]

date_list = [dt.datetime(
    date_df.loc[_][1],
    date_df.loc[_][2],
    date_df.loc[_][3],
    date_df.loc[_][4],
    date_df.loc[_][5]
) for _ in date_df.index]

datas_df = pd.DataFrame(columns=['date'])
datas_df['date'] = date_list

for col in household_df:
    household_name = "{}-{}-{}".format(
        household_df[col][0],
        household_df[col][1],
        household_df[col][2]
    )
    datas_df[household_name] = household_df[col][3:].to_list()
    
datas_df = datas_df.replace("-", 0)

sum_df = pd.DataFrame(columns=['date', 'kWh', 'kW'])
sum_df['date'] = date_list
sum_df['kWh'] = [round (_) for _ in datas_df[datas_df.columns.difference(['date'])].sum(axis=1).to_list()]
sum_df['kW'] = (sum_df['kWh'] / 0.25).to_list()

peak_df = pd.DataFrame(columns=['month', 'peak (kW)'])
for month in range(1,13):
    peak_df = peak_df.append({
        "month": str(month),
        "peak (kW)": sum_df[sum_df['date'].dt.month == month]['kW'].max()
    }, ignore_index=True)

month_usage_df = pd.DataFrame(columns=['month'])
month_usage_df['month'] = [_ for _ in range(1,13)]

for name in datas_df[datas_df.columns.difference(['date'])]:
    self_household_df = datas_df[['date',name]].copy()
    
    month_usage_df[name] = [
        round(self_household_df[self_household_df['date'].dt.month == month][name].sum())
        for month in range(1,13)
    ]

analysis_df = month_usage_df.set_index("month")
better_comp_df = pd.DataFrame()

for month in range(1, 13):
    # 1. 월별 사용량 데이터 파싱
    month_datas_df = pd.DataFrame(columns=["name","usage (kWh)"])
    
    for idx in analysis_df.loc[month].index:
        household_name = idx
        household_kWh = analysis_df.loc[month][idx]

        month_datas_df = month_datas_df.append({
            "name": household_name,
            "usage (kWh)": household_kWh
        }, ignore_index=True)

    # Thinking
    # 세대부 전기는 정해져 있는데,
    # 공용부 전기는 정해져 있지가 않아서 입력되는 percentage에 따라, 변화하도록

    # 다음과 같은 공식을 사용할 수 있다.
    # n -> 공용부가 전체 APT에서 차지할 percentage
    # APT : households_kWh = 100 : (100 - n)
    # APT : public_kWh = 100 : n

    # 이에 따라,
    # APT = (households_kWh * 100) / (100 - n)
    # public_kwh = APT - households_kwh
    min_per = 10
    max_per = 80
    rows = np.array([])

    for PUBLIC_PERCENTAGE in range(min_per, max_per + 1):
        households_kWh = sum(month_datas_df['usage (kWh)'].values)
        APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
        public_kWh = round(APT - households_kWh)
        in_month = month
        
        if month == 13:
            in_month = 1

        # 종합계약
        calc = ManagementOffice(
            month=in_month,
            peaks=peak_df, 
            households=month_datas_df,
            APT=APT,
            contract="종합계약",
            general_fee_info=['고압 A', 1]
        )

        # 단일계약
        single_calc = ManagementOffice(
            month=in_month,
            peaks=peak_df, 
            households=month_datas_df,
            APT=APT,
            contract="단일계약"
        )

        cnt = len(calc.households)
        comp_cnt = 0
        draw_cnt = 0
        single_cnt = 0
        for idx in range(0, cnt):
            if calc.households[idx].bill > single_calc.households[idx].bill:
                single_cnt += 1
            elif calc.households[idx].bill < single_calc.households[idx].bill:
                comp_cnt += 1
            else:
                draw_cnt

        rows = np.append(rows,comp_cnt)

        IPython.display.clear_output()

    better_comp_df = better_comp_df.append(
        pd.Series(rows, index=["{}".format(_) for _ in range(min_per, max_per + 1)], name=month))
    better_comp_df
    
better_comp_df

,10,11,12,13,14,15,16,17,18,19,...,71,72,73,74,75,76,77,78,79,80
1,11.0,13.0,15.0,18.0,18.0,19.0,22.0,24.0,35.0,44.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
2,13.0,16.0,18.0,20.0,21.0,28.0,37.0,41.0,52.0,60.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
3,52.0,70.0,82.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
4,65.0,76.0,93.0,113.0,113.0,113.0,113.0,113.0,113.0,113.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
6,4.0,5.0,5.0,6.0,8.0,8.0,9.0,12.0,18.0,19.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
9,62.0,81.0,95.0,114.0,137.0,137.0,137.0,137.0,137.0,137.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
10,57.0,73.0,88.0,103.0,115.0,115.0,115.0,115.0,115.0,115.0,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0


In [36]:
sim_test = pd.DataFrame(columns=['차원축소 사이즈', '추천 인덱스'])

matrix = np.array(better_comp_df)
r, c = matrix.shape
cnt = c

while cnt >= 2:
    dr_matrix = None
    if cnt == c:
        dr_matrix = matrix
    else:
        SVD = TruncatedSVD(n_components=cnt)
        dr_matrix = SVD.fit_transform(matrix)
        
    imp = np.array([])
    for A in dr_matrix:
        _ = np.array([])
        for B in dr_matrix:
            _ = np.append(_, improved_similarity(
                A, B, 0.99
            ))
            
        imp = np.append(imp, _)
        
    reco_idx = imp.reshape(-1,r).sum(axis=1).argmax()
    sim_test = sim_test.append({
        "차원축소 사이즈": cnt,
        "추천 인덱스": reco_idx
    }, ignore_index=True)
    
    cnt-=1
    
sim_test

,차원축소 사이즈,추천 인덱스
0,71,0
1,70,9
2,69,9
3,68,9
4,67,9
...,...,...
65,6,9
66,5,9
67,4,3
68,3,8


In [63]:
def euclidean_distance(A, B):
    err = 0
    length = len(A)
    for idx in range(0, length):
        tmp = ((A[idx] - B[idx]) ** 2)
        err += tmp
        
    return mt.sqrt(err)

@jit
def cosine_similarity(A,B):
    return dot(A, B) / (norm(A) * norm(B))

@jit
def sumDiffer(A,B):
    length = len(A)
    err = 0
    for idx in range(0, length):
        err += ((A[idx] - B[idx]) ** 2)
    return mt.sqrt(err / length)

@jit
def improved_similarity(A, B, w):
    cos_sim = cosine_similarity(A, B)
    sum_diff = sumDiffer(A, B)
    w **= sum_diff
    
    return cos_sim * w

In [64]:
@jit
def analysis_reco(dr_matrix):
    imp_weight = 0.99
    results = np.array([])

    # 2. check similarity, improved cosine similarity
    for A in dr_matrix:
        _ = np.array([])
        for B in dr_matrix:
            _ = np.append(_, improved_similarity(A, B, imp_weight))
        results = np.append(results, _)

    results = results.reshape(r, -1)

    sum_results = results.sum(axis=0)
    max_sim_idx = sum_results.argmax()
    
    return max_sim_idx


def analysis(matrix):
    cols = ['차원축소 사이즈', '추천 인덱스', '유클리디안 거리', '코사인 유사도', '피어슨 상관계수', '향상된 코사인 유사도']
    info_df = pd.DataFrame(columns=cols)
    reco_df = pd.DataFrame(columns=["차원축소 사이즈", "추천 인덱스"])

    r,c = matrix.shape
    dr_size = c
#     norm_matrix = (matrix.flatten() - matrix.flatten().min()) / (matrix.flatten().max() - matrix.flatten().min(axis=0))
#     norm_matrix = norm_matrix.reshape(r, c)
    recos_matrix = np.array([])

    while dr_size >= 2:
        r, c = matrix.shape
        dr_matrix = None

        if dr_size == c:
            dr_size = c
            dr_matrix = matrix 
        else:
            # 1. 차원 축소 진행
            SVD = TruncatedSVD(n_components=dr_size)
            dr_matrix = SVD.fit_transform(matrix)
            
        reco_df = reco_df.append({
            "차원축소 사이즈": dr_size,
            "추천 인덱스":analysis_reco(dr_matrix)
        })
        dr_size -= 1

    for idx in reco_df.index:
        sim_cols = ['유클리디안 거리', '코사인 유사도', '피어슨 상관계수', '향상된 코사인 유사도']
        sim_df = pd.DataFrame(columns=cols)

        dr_size = reco_df.iloc[idx]['차원축소 사이즈']
        reco_idx = reco_df.iloc[idx]['추천 인덱스']

        A = matrix[reco_idx]
        A_norm = norm_matrix[reco_idx]
        B_values = np.delete(matrix, reco_idx, axis=0)
        B_values_norm = np.delete(norm_matrix, reco_idx, axis=0)

        for idx,B in enumerate(B_values):
            sim_df = sim_df.append({
                "유클리디안 거리": euclidean_distance(A, B),
                "코사인 유사도": cosine_similarity(A,B),
                "피어슨 상관계수": stats.pearsonr(A,B)[0],
                "향상된 코사인 유사도": improved_similarity(A_norm, B_values_norm[idx], 0.99)
            }, ignore_index=True)

        chk = info_df[(info_df['추천 인덱스'] == reco_idx)]
        if len(chk) == 0:
            info_df = info_df.append({
                "차원축소 사이즈": dr_size,
                "추천 인덱스": reco_idx,
                "유클리디안 거리": sim_df['유클리디안 거리'].mean(),
                "코사인 유사도": sim_df['코사인 유사도'].mean(),
                "피어슨 상관계수": sim_df['피어슨 상관계수'].mean(),
                "향상된 코사인 유사도": sim_df['향상된 코사인 유사도'].mean()
            }, ignore_index=True)

            recos_matrix = np.append(recos_matrix, A)


    recos_matrix = recos_matrix.reshape(-1,c)
    reco_matrix = np.round(recos_matrix.mean(axis=0))

    reco_idx = len(matrix)
    matrix_in_reco = np.append(matrix, reco_matrix).reshape(-1, c)
    norm_matrix = (matrix_in_reco.flatten() - matrix_in_reco.flatten().min()) / (matrix_in_reco.flatten().max() - matrix_in_reco.flatten().min(axis=0))
    norm_matrix = norm_matrix.reshape(-1, c)
    A = matrix_in_reco[reco_idx]
    A_norm = norm_matrix[reco_idx]
    B_values = np.delete(matrix_in_reco, reco_idx, axis=0)
    B_values_norm = np.delete(norm_matrix, reco_idx, axis=0)

    for idx,B in enumerate(B_values):
        sim_df = sim_df.append({
            "유클리디안 거리": euclidean_distance(A, B),
            "코사인 유사도": cosine_similarity(A,B),
            "피어슨 상관계수": stats.pearsonr(A,B)[0],
            "향상된 코사인 유사도": improved_similarity(A_norm, B_values_norm[idx], 0.99)
        }, ignore_index=True)

    info_df = info_df.append({
                "차원축소 사이즈": "sims mean",
                "추천 인덱스": "sims mean",
                "유클리디안 거리": sim_df['유클리디안 거리'].mean(),
                "코사인 유사도": sim_df['코사인 유사도'].mean(),
                "피어슨 상관계수": sim_df['피어슨 상관계수'].mean(),
                "향상된 코사인 유사도": sim_df['향상된 코사인 유사도'].mean()
            }, ignore_index=True)

    # 평균 넣기
    mean_matrix = np.round(matrix.mean(axis=0))
    mean_idx = len(matrix)
    matrix_in_mean = np.append(matrix, mean_matrix).reshape(-1, c)
    norm_matrix = (matrix_in_mean.flatten() - matrix_in_mean.flatten().min()) / (matrix_in_mean.flatten().max() - matrix_in_mean.flatten().min(axis=0))
    norm_matrix = norm_matrix.reshape(-1, c)
    A = matrix_in_mean[mean_idx]
    A_norm = norm_matrix[mean_idx]
    B_values = np.delete(matrix_in_mean, mean_idx, axis=0)
    B_values_norm = np.delete(norm_matrix, mean_idx, axis=0)
    
    for idx,B in enumerate(B_values):
            sim_df = sim_df.append({
                "유클리디안 거리": euclidean_distance(A, B),
                "코사인 유사도": cosine_similarity(A,B),
                "피어슨 상관계수": stats.pearsonr(A,B)[0],
                "향상된 코사인 유사도": improved_similarity(A_norm, B_values_norm[idx], 0.99)
            }, ignore_index=True)

    info_df = info_df.append({
                "차원축소 사이즈": "mean",
                "추천 인덱스": "mean",
                "유클리디안 거리": sim_df['유클리디안 거리'].mean(),
                "코사인 유사도": sim_df['코사인 유사도'].mean(),
                "피어슨 상관계수": sim_df['피어슨 상관계수'].mean(),
                "향상된 코사인 유사도": sim_df['향상된 코사인 유사도'].mean()
            }, ignore_index=True)
        
        
    return (recos_matrix,info_df)

In [65]:
# 15분 데이터
timeslot_size = 96
month_anal_pattern_df = pd.DataFrame()
month_anal_usage_df = pd.DataFrame()

for name in tqdm(datas_df[datas_df.columns.difference(['date'])].columns):
    self_household_df = datas_df[['date',name]].copy()
    month_kwh_np = np.array([])
    anal_kwh_np = np.array([])
    days = 31
    month = 1
    
    anal_np = np.array(self_household_df[self_household_df['date'].dt.month == month][name].copy()).reshape(-1, timeslot_size)
    anal_m = np.array([])

    for a in anal_np:
        if len(set(a)) != 1:
            anal_m = np.append(anal_m, a)
    anal_m = anal_m.reshape(-1, timeslot_size)
    (r_m, info) = analysis(anal_m * 1000)
    sim_anal = r_m.mean(axis=0) / 1000
        
    month_anal_usage_df[name] = [
        round(sim_anal.sum() * 31)
    ]
    month_anal_pattern_df[name] = sim_anal
    
month_anal_usage_df

  0%|          | 0/398 [00:00<?, ?it/s]<ipython-input-64-3840aefd39eb>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "analysis_reco" failed type inference due to: Undecided type $80load_method.1 := <undecided>
During: resolving caller type: $80load_method.1
During: typing of call at <ipython-input-64-3840aefd39eb> (13)


File "<ipython-input-64-3840aefd39eb>", line 13:
def analysis_reco(dr_matrix):
    <source elided>

    results = results.reshape(r, -1)
    ^

  @jit
<ipython-input-64-3840aefd39eb>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "analysis_reco" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-64-3840aefd39eb>", line 7:
def analysis_reco(dr_matrix):
    <source elided>
    # 2. check similarity, improved cosine similarity
    for A in dr_matrix:
    ^

  @jit
/Users/formegusto/op

ValueError: attempt to get argmax of an empty sequence

In [66]:
analysis_reco(matrix)

<ipython-input-64-3840aefd39eb>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "analysis_reco" failed type inference due to: Undecided type $80load_method.1 := <undecided>
During: resolving caller type: $80load_method.1
During: typing of call at <ipython-input-64-3840aefd39eb> (13)


File "<ipython-input-64-3840aefd39eb>", line 13:
def analysis_reco(dr_matrix):
    <source elided>

    results = results.reshape(r, -1)
    ^

  @jit
<ipython-input-64-3840aefd39eb>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "analysis_reco" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-64-3840aefd39eb>", line 7:
def analysis_reco(dr_matrix):
    <source elided>
    # 2. check similarity, improved cosine similarity
    for A in dr_matrix:
    ^

  @jit
/Users/formegusto/opt/anaconda3/lib/python3.8/site-packages

ValueError: attempt to get argmax of an empty sequence